# Report on the Neural Network Model

**Overview of the analysis**: Explain the purpose of this analysis.

**Results**: Using bulleted lists and images to support your answers, address the following questions.

**Data Preprocessing**

What variable(s) are the target(s) for your model?
What variable(s) are the features for your model?
What variable(s) should be removed from the input data because they are neither targets nor features?

**Compiling, Training, and Evaluating the Model**

How many neurons, layers, and activation functions did you select for your neural network model, and why?
Were you able to achieve the target model performance?
What steps did you take in your attempts to increase model performance?

**Summary**: Summarize the overall results of the deep learning model. Include a recommendation for how a different model could solve this classification problem, and then explain your recommendation.

In [ ]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf

pd.options.display.max_rows = 500
pd.options.display.max_columns = 50

In [ ]:
#  Import and read the charity_data.csv

df = pd.read_csv("Resources/charity_data.csv")
print(df.info())
df.head()

In [ ]:
df.hist(figsize = (15,15))

In [ ]:
sns.PairGrid(df).map_upper(plt.scatter) 

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

df = df.drop(columns=["EIN", "NAME"])

In [ ]:
# Determine the number of unique values in each column.

for col in df.columns:
    print(f"{col} \n{df[col].value_counts()}\n\n")